In [2]:
# reload magics
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datasets
from project_modules.io import load_dataset_to_df
from project_modules.classifcation import classify_MP,getXY, boruta_fs
from sklearn.ensemble import RandomForestClassifier
import multiprocessing
from sklearn.model_selection import cross_val_score

# from project_modules.utils import get_logger
# logger = get_logger("log-data-combine-split.log")
# # read the parameter file

# from project_modules.utils import read_parameters
# parms = read_parameters("/Users/david/projects/lc-project-data/project.yaml")

In [3]:
# from pathlib import Path
# input_dir = Path(parms["dir_proc_data"])
# ipath = Path(input_dir / "SYNTHETIC_dataset_dict")
# ipath

# Import Data

In [8]:
T81_tr = load_dataset_to_df("../Data/DataV1/TTS/T81_Train.arrow", verbose=True)
T81_ts = load_dataset_to_df("../Data/DataV1/TTS/T81_Test.arrow", verbose=True)
T81_h = load_dataset_to_df("../Data/DataV1/TTS/T81_Holdout.arrow", verbose=True)
T81_tr.shape, T81_ts.shape, T81_h.shape

((2842, 111), (948, 111), (948, 111))

In [9]:
T83_tr = load_dataset_to_df( "../Data/DataV1/TTS/T83_Train.arrow", verbose=True)
T83_ts = load_dataset_to_df("../Data/DataV1/TTS/T83_Test.arrow", verbose=True)
T83_h = load_dataset_to_df("../Data/DataV1/TTS/T83_Holdout.arrow", verbose=True)
T83_tr.shape, T83_ts.shape, T83_h.shape

((3315, 90), (1105, 90), (1106, 90))

In [10]:
T81_tr

,sex,age,dag,MED-Analgesics,MED-Anesthetics,MED-Antibacterials,MED-Anticonvulsants,MED-Antidepressants,MED-Antifungals,MED-Antimigraine-agents,...,HHX-OTHER-surgery,HHX-OTHER-ulcerative-colitis,HHX-Obesity,HHX-Other,HHX-Pregnant,SUBST-1,SUBST-5,LC_STATUS,__index_level_0__,LC_STATUS_DAG
0,2.0,59.0,2,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,1.0,0.0,LC_NEG,1364,LC_NEG_2
1,1.0,32.0,4,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,LC_POS,3561,LC_POS_4
2,2.0,57.0,2,0,0,0,0,0,0,0,...,0,0,0.0,1.0,0.0,0.0,0.0,LC_NEG,2409,LC_NEG_2
3,1.0,51.0,4,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,LC_POS,3102,LC_POS_4
4,1.0,72.0,4,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,LC_POS,3333,LC_POS_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2837,1.0,65.0,2,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,1.0,0.0,LC_NEG,2284,LC_NEG_2
2838,2.0,28.0,2,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,LC_NEG,2651,LC_NEG_2
2839,1.0,86.0,2,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,LC_NEG,1268,LC_NEG_2
2840,1.0,41.0,2,0,0,0,0,0,0,0,...,0,0,1.0,1.0,0.0,1.0,0.0,LC_NEG,272,LC_NEG_2


In [11]:
lDataFrames = [T81_tr, T81_ts, T81_h, T83_tr, T83_ts, T83_h]
lColDrop = ["__index_level_0__", "LC_STATUS_DAG"]

for df in lDataFrames:
    df.drop(df[df['LC_STATUS']=='HC'].index, inplace=True) # drop HC
    df['LC_STATUS'] = df['LC_STATUS'].apply(lambda x: 1 if x == 'LC_POS' else 0) #Convert to 0==LCNeg, 1==LCPos
    df.drop(columns=lColDrop, inplace=True) # drop unneeded columns

In [13]:
T81_tr.shape,T83_tr.shape

((2794, 109), (3267, 88))

In [21]:
lScorersBinary = [
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    # "average_precision",
    "NPV",
    "PPV",
    # "neg_mean_squared_error",
]

# Basic RF Classifier - Holdout sets, all Features

In [11]:

lClassifiers = ["RF", "ET", "LogR"] # different classifiers
lCV = [3,5,10] # different cross-validation splits
lTrees=[100,1000,10000] #num trees
lMD=[None] #max depth

## T83 - Holdout

In [8]:
x,y = getXY(T83_h)

In [9]:
dfResults = classify_MP(X=x, y=y, lScorers=lScorersBinary, lClassifiers=lClassifiers, lTrees=lTrees, lMD=lMD,lCV=lCV)

In [10]:
# dfResults.columns

In [11]:
lResCol = ['Title','cv']+[x for x in dfResults.columns if x.startswith("param_") or x.startswith("mean_test_")]

In [12]:
dfResults['Title']='RT83_Holdout_LCNeg_LCPos'

/tmp/ipykernel_38070/3276964138.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfResults['Title']='RT83_Holdout_LCNeg_LCPos'


In [13]:
dfResults[lResCol]

,Title,cv,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,mean_test_accuracy,mean_test_balanced_accuracy,mean_test_roc_auc,mean_test_f1,mean_test_recall,mean_test_sensitivity,mean_test_specificity,mean_test_precision,mean_test_NPV,mean_test_PPV
0,RT83_Holdout_LCNeg_LCPos,3,RandomForestClassifier(),None,100,42,NaN,0.719008,0.697772,0.769934,0.623408,0.533755,0.533755,0.861789,0.750168,0.705580,0.750168
1,RT83_Holdout_LCNeg_LCPos,3,RandomForestClassifier(),None,1000,42,NaN,0.719008,0.697772,0.771349,0.623334,0.533755,0.533755,0.861789,0.750956,0.705630,0.750956
2,RT83_Holdout_LCNeg_LCPos,3,RandomForestClassifier(),None,10000,42,NaN,0.715335,0.694520,0.772522,0.620583,0.533755,0.533755,0.855285,0.743386,0.703864,0.743386
3,RT83_Holdout_LCNeg_LCPos,3,ExtraTreesClassifier(),None,100,42,NaN,0.707071,0.690347,0.760060,0.625444,0.561181,0.561181,0.819512,0.708421,0.707675,0.708421
4,RT83_Holdout_LCNeg_LCPos,3,ExtraTreesClassifier(),None,1000,42,NaN,0.707989,0.691160,0.761295,0.625804,0.561181,0.561181,0.821138,0.708182,0.708346,0.708182
5,RT83_Holdout_LCNeg_LCPos,3,ExtraTreesClassifier(),None,10000,42,NaN,0.707071,0.690347,0.761305,0.625048,0.561181,0.561181,0.819512,0.706659,0.707954,0.706659
6,RT83_Holdout_LCNeg_LCPos,3,LogisticRegression(),NaN,NaN,NaN,1000,0.651974,0.639632,0.695570,0.575298,0.544304,0.544304,0.734959,0.613169,0.677719,0.613169
0,RT83_Holdout_LCNeg_LCPos,5,RandomForestClassifier(),None,100,42,NaN,0.713491,0.692608,0.760451,0.616849,0.531557,0.531557,0.853659,0.737333,0.703222,0.737333
1,RT83_Holdout_LCNeg_LCPos,5,RandomForestClassifier(),None,1000,42,NaN,0.714413,0.694630,0.759707,0.622592,0.542105,0.542105,0.847154,0.733420,0.706186,0.733420
2,RT83_Holdout_LCNeg_LCPos,5,RandomForestClassifier(),None,10000,42,NaN,0.715326,0.695682,0.759314,0.624289,0.544211,0.544211,0.847154,0.733931,0.707118,0.733931


In [14]:
dfResults.to_csv("OUTPUT/MP/05-classifiers/CA_T83_Holdout_CV3510.csv", index=False)
dfResults[lResCol].to_csv("OUTPUT/MP/05-classifiers/CA_T83_Holdout_CV3510_selCol.csv", index=False)

## T81 - Holdout

In [15]:
x,y = getXY(T81_h)

In [16]:
dfResults = classify_MP(X=x, y=y, lScorers=lScorersBinary, lClassifiers=lClassifiers, lTrees=lTrees, lMD=lMD,lCV=lCV)

In [17]:
# dfResults.columns

In [18]:
lResCol = ['Title','cv']+[x for x in dfResults.columns if x.startswith("param_") or x.startswith("mean_test_")]

In [19]:
dfResults['Title']='RT81_Holdout_LCNeg_LCPos'

/tmp/ipykernel_38070/1820217221.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfResults['Title']='RT81_Holdout_LCNeg_LCPos'


In [20]:
dfResults[lResCol]

,Title,cv,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,mean_test_accuracy,mean_test_balanced_accuracy,mean_test_roc_auc,mean_test_f1,mean_test_recall,mean_test_sensitivity,mean_test_specificity,mean_test_precision,mean_test_NPV,mean_test_PPV
0,RT81_Holdout_LCNeg_LCPos,3,RandomForestClassifier(),None,100,42,NaN,0.777648,0.773656,0.873534,0.749595,0.719341,0.719341,0.827971,0.783287,0.774162,0.783287
1,RT81_Holdout_LCNeg_LCPos,3,RandomForestClassifier(),None,1000,42,NaN,0.772292,0.768335,0.873674,0.743736,0.714711,0.714711,0.821958,0.776162,0.770174,0.776162
2,RT81_Holdout_LCNeg_LCPos,3,RandomForestClassifier(),None,10000,42,NaN,0.772295,0.768662,0.873714,0.744804,0.719357,0.719357,0.817966,0.773597,0.772534,0.773597
3,RT81_Holdout_LCNeg_LCPos,3,ExtraTreesClassifier(),None,100,42,NaN,0.766916,0.762836,0.862253,0.737534,0.707718,0.707718,0.817954,0.770794,0.764747,0.770794
4,RT81_Holdout_LCNeg_LCPos,3,ExtraTreesClassifier(),None,1000,42,NaN,0.766909,0.763345,0.864492,0.739311,0.714711,0.714711,0.811978,0.766542,0.767962,0.766542
5,RT81_Holdout_LCNeg_LCPos,3,ExtraTreesClassifier(),None,10000,42,NaN,0.765840,0.762486,0.864631,0.739173,0.717010,0.717010,0.807962,0.763118,0.768259,0.763118
6,RT81_Holdout_LCNeg_LCPos,3,LogisticRegression(),NaN,NaN,NaN,1000,0.758310,0.754640,0.833568,0.729847,0.705274,0.705274,0.804006,0.756527,0.759983,0.756527
0,RT81_Holdout_LCNeg_LCPos,5,RandomForestClassifier(),None,100,42,NaN,0.784072,0.780397,0.874915,0.758158,0.730794,0.730794,0.830000,0.788361,0.781546,0.788361
1,RT81_Holdout_LCNeg_LCPos,5,RandomForestClassifier(),None,1000,42,NaN,0.784095,0.781414,0.876196,0.761900,0.744828,0.744828,0.818000,0.780777,0.787887,0.780777
2,RT81_Holdout_LCNeg_LCPos,5,RandomForestClassifier(),None,10000,42,NaN,0.783020,0.780577,0.876005,0.761254,0.747153,0.747153,0.814000,0.776896,0.789016,0.776896


In [21]:
dfResults.to_csv("OUTPUT/MP/05-classifiers/CA_T81_Holdout_CV3510.csv", index=False)
dfResults[lResCol].to_csv("OUTPUT/MP/05-classifiers/CA_T81_Holdout_CV3510_selCol.csv", index=False)

## Summary
T81 Metrics Superior to T83... likely some important columns there

Main improvmeent in sensitivity/recall

In [5]:
lResCol = [
    "Title",
    "cv",
    "param_clf",
    "param_clf__max_depth",
    "param_clf__n_estimators",
    "param_clf__random_state",
    # "param_clf__max_iter",
    "mean_test_accuracy",
    "mean_test_balanced_accuracy",
    "mean_test_roc_auc",
    "mean_test_f1",
    "mean_test_recall",
    "mean_test_sensitivity",
    "mean_test_specificity",
    "mean_test_precision",
    "mean_test_NPV",
    "mean_test_PPV",
]

In [7]:
rT81 = pd.read_csv("OUTPUT/MP/05-classifiers/CA_T81_Holdout_CV3510.csv")
rT83 = pd.read_csv("OUTPUT/MP/05-classifiers/CA_T83_Holdout_CV3510.csv")
dfR = pd.concat([rT81,rT83])[lResCol]
dfR[(dfR['cv']==5) & (dfR['param_clf']=='RandomForestClassifier()') & (dfR['param_clf__n_estimators']==1000)].T

,8,8
Title,RT81_Holdout_LCNeg_LCPos,RT83_Holdout_LCNeg_LCPos
cv,5,5
param_clf,RandomForestClassifier(),RandomForestClassifier()
param_clf__max_depth,NaN,NaN
param_clf__n_estimators,1000.0,1000.0
param_clf__random_state,42.0,42.0
mean_test_accuracy,0.784095,0.714413
mean_test_balanced_accuracy,0.781414,0.69463
mean_test_roc_auc,0.876196,0.759707
mean_test_f1,0.7619,0.622592


# Boruta FS

In [105]:
## Below runs all permutations of different datasets of interest at different parameters -- saves featuresets externally
# Completed Runs:
# lMD = [3,5,7]
# lTitle = ['T81','T83','T81noDag','T83noDat']
# lDatasets=[T81_tr,T83_tr,T81_tr.drop(columns=['dag']),T83_tr.drop(columns=['dag'])]

lMD = [3,5]
lTitle = ['T81noDag','T83noDat']
lDatasets=[T81_tr.drop(columns=['dag']),T83_tr.drop(columns=['dag'])]

for i, ds in enumerate(lDatasets):
    x, y = getXY(ds)
    for md in lMD:
        print(f'------{lTitle[i]}, md={md}')
        _, _ = boruta_fs(X=x.values, y=y,feat_list=x.columns, trees=1000,ittr=500,threshold=100,top_rank=5,verbose=0,model=RandomForestClassifier(n_jobs=-1,class_weight='balanced',max_depth=md,random_state=42),fileName=f"OUTPUT/MP/05-classifiers/FS/FS_{lTitle[i]}_Boruta_T1000_itrr500_thres100_topR5_MD{md}.csv")

------T81noDag, 3
1	True	age
1	True	MED-Cardiovascular-agents
1	True	SYMPT-abdominal_pain___2
5	False	SYMPT-chest_pain___1
1	True	SYMPT-chest_pain___2
1	True	SYMPT-confusion_altered_mental_s___2
1	True	SYMPT-cough___1
1	True	SYMPT-cough___2
1	True	SYMPT-diarrhea___2
1	True	SYMPT-dizziness___1
1	True	SYMPT-dizziness___2
1	True	SYMPT-extremity_weakness_or_numb___1
1	True	SYMPT-extremity_weakness_or_numb___2
1	True	SYMPT-fatigue___1
1	True	SYMPT-fatigue___2
1	True	SYMPT-fever_38_0oc___1
1	True	SYMPT-fever_38_0oc___2
1	True	SYMPT-headache___1
1	True	SYMPT-headache___2
1	True	SYMPT-joint_pain_arthralgia___2
1	True	SYMPT-leg_swelling_edema___2
1	True	SYMPT-loss_of_appetite___1
1	True	SYMPT-loss_of_appetite___2
2	False	SYMPT-loss_of_taste_lost_of_smel___1
1	True	SYMPT-loss_of_taste_lost_of_smel___2
1	True	SYMPT-muscle_aches_myalgia___1
1	True	SYMPT-muscle_aches_myalgia___2
1	True	SYMPT-nausea_vomiting___2
1	True	SYMPT-shortness_of_breath_dyspne___1
1	True	SYMPT-shortness_of_breath_dyspne___2


# Test Boruta Produced Feature Sets

In [16]:
import glob

folder_path = 'OUTPUT/MP/05-classifiers/FS'  # Replace with the actual folder of boruta exported features

csv_files = glob.glob(folder_path + '/*Boruta*.csv')

# Get all featuresets identified and save them to dFeat
dFeat = {}
for file in csv_files:
    t = file.replace(folder_path + '/FS_','').replace('.csv','')
    dfTmp = pd.read_csv(file,names=['rank','trueFeat','feat'])
    trueFeatTmp = dfTmp[dfTmp["trueFeat"] == True]['feat'].to_list()
    topFeatTmp = dfTmp["feat"].to_list()
    dFeat[t+'_True']=trueFeatTmp
    dFeat[t+'_Top']=topFeatTmp
    print(t)


T83noDag_Boruta_T1000_itrr200_thres100_topR5_MD7
T81_Boruta_T1000_itrr200_thres100_topR5_MD7
T81noDag_Boruta_T1000_itrr200_thres100_topR5_MD7
T81noDag_Boruta_T1000_itrr500_thres100_topR5_MD3
T83noDat_Boruta_T1000_itrr500_thres100_topR5_MD5
T81_Boruta_T1000_itrr200_thres100_topR5_MD3
T81noDag_Boruta_T1000_itrr500_thres100_topR5_MD5
T83noDag_Boruta_T1000_itrr200_thres100_topR5_MD3
T83noDat_Boruta_T1000_itrr500_thres100_topR5_MD3
T83_Boruta_T1000_itrr200_thres100_topR5_MD7
T83_Boruta_T1000_itrr200_thres100_topR5_MD3


In [17]:
# Function runs a simple test of ideal parameteres T=1000, RF, MD=None, cv=5
def test_reduced_features(df,featRedu,title):
    x,y = getXY(df)
    x = x[featRedu]
    dfRTmp = classify_MP(
        X=x,
        y=y,
        lScorers=lScorersBinary,
        lClassifiers=["RF"],
        lTrees=[1000],
        lMD=[None],
        lCV=[5],
    )
    dfRTmp["Title"] = f"Test_{title}_LCN_LCP"
    return dfRTmp
def test_reduced_features_MP(inputTuple):
    df,featRedu,title = inputTuple
    x,y = getXY(df)
    x = x[featRedu]
    dfRTmp = classify_MP(
        X=x,
        y=y,
        lScorers=lScorersBinary,
        lClassifiers=["RF"],
        lTrees=[1000],
        lMD=[None],
        lCV=[5],
    )
    dfRTmp["Title"] = f"Test_{title}_LCN_LCP"
    return dfRTmp

In [18]:
from tqdm.contrib.concurrent import process_map

In [19]:
lResCol = [
    "Title",
    "cv",
    "param_clf",
    "param_clf__max_depth",
    "param_clf__n_estimators",
    "param_clf__random_state",
    # "param_clf__max_iter",
    "mean_test_accuracy",
    "mean_test_balanced_accuracy",
    "mean_test_roc_auc",
    "mean_test_f1",
    "mean_test_recall",
    "mean_test_sensitivity",
    "mean_test_specificity",
    "mean_test_precision",
    "mean_test_NPV",
    "mean_test_PPV",
]

In [22]:
# Go through all featuresets identified and determine metrics
lTestResults = []
lArgs = []
for k,v in dFeat.items():
    if 'T81' in k:
        dataset=T81_ts
        # print('T81',v,k)
    elif 'T83' in k:
        dataset=T83_ts
        # print('T83',v,k)
    else:
        raise Exception("Unknown dataset")
    lArgs.append((dataset,v,k))

with multiprocessing.Pool(20) as pool:
    lTestResults = pool.map(test_reduced_features_MP, lArgs)
    # lTestResults.append(test_reduced_features(dataset,v,k))


/home/mtrx/miniconda3/envs/clust/lib/python3.11/site-packages/joblib/parallel.py:1332: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
/home/mtrx/miniconda3/envs/clust/lib/python3.11/site-packages/joblib/parallel.py:1332: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
/home/mtrx/miniconda3/envs/clust/lib/python3.11/site-packages/joblib/parallel.py:1332: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
/home/mtrx/miniconda3/envs/clust/lib/python3.11/site-packages/joblib/parallel.py:1332: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, par

In [28]:
pd.set_option("display.max_colwidth", None)
dfTestResults = pd.concat(lTestResults)[lResCol].sort_values(
    "mean_test_accuracy", ascending=False
)
dfTestResults

,Title,cv,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,mean_test_accuracy,mean_test_balanced_accuracy,mean_test_roc_auc,mean_test_f1,mean_test_recall,mean_test_sensitivity,mean_test_specificity,mean_test_precision,mean_test_NPV,mean_test_PPV
0,Test_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True_LCN_LCP,5,RandomForestClassifier(),None,1000,42,0.770416,0.766269,0.817495,0.741037,0.708982,0.708982,0.823556,0.778037,0.766453,0.778037
0,Test_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_Top_LCN_LCP,5,RandomForestClassifier(),None,1000,42,0.759692,0.758607,0.843298,0.741805,0.743598,0.743598,0.773616,0.740448,0.777003,0.740448
0,Test_T81noDag_Boruta_T1000_itrr500_thres100_topR5_MD3_True_LCN_LCP,5,RandomForestClassifier(),None,1000,42,0.729630,0.725180,0.817960,0.694367,0.662764,0.662764,0.787596,0.732189,0.730064,0.732189
0,Test_T81_Boruta_T1000_itrr200_thres100_topR5_MD3_True_LCN_LCP,5,RandomForestClassifier(),None,1000,42,0.729630,0.725180,0.817960,0.694367,0.662764,0.662764,0.787596,0.732189,0.730064,0.732189
0,Test_T81_Boruta_T1000_itrr200_thres100_topR5_MD3_Top_LCN_LCP,5,RandomForestClassifier(),None,1000,42,0.729625,0.725342,0.816219,0.695202,0.665170,0.665170,0.785515,0.731783,0.730938,0.731783
0,Test_T81noDag_Boruta_T1000_itrr500_thres100_topR5_MD3_Top_LCN_LCP,5,RandomForestClassifier(),None,1000,42,0.729625,0.725342,0.816219,0.695202,0.665170,0.665170,0.785515,0.731783,0.730938,0.731783
0,Test_T83_Boruta_T1000_itrr200_thres100_topR5_MD3_Top_LCN_LCP,5,RandomForestClassifier(),None,1000,42,0.726343,0.713939,0.774948,0.662521,0.616842,0.616842,0.811036,0.717103,0.732729,0.717103
0,Test_T83_Boruta_T1000_itrr200_thres100_topR5_MD3_True_LCN_LCP,5,RandomForestClassifier(),None,1000,42,0.714413,0.701439,0.763923,0.646121,0.600000,0.600000,0.802879,0.703620,0.722582,0.703620
0,Test_T81noDag_Boruta_T1000_itrr500_thres100_topR5_MD5_Top_LCN_LCP,5,RandomForestClassifier(),None,1000,42,0.709269,0.706220,0.803956,0.678488,0.662924,0.662924,0.749515,0.698305,0.720643,0.698305
0,Test_T83_Boruta_T1000_itrr200_thres100_topR5_MD7_Top_LCN_LCP,5,RandomForestClassifier(),None,1000,42,0.702448,0.689859,0.757975,0.634234,0.591579,0.591579,0.788138,0.687782,0.713922,0.687782


In [31]:
dfTestResults[['Title']+[x for x in dfTestResults.columns if x.startswith("mean_test_")]]

,Title,mean_test_accuracy,mean_test_balanced_accuracy,mean_test_roc_auc,mean_test_f1,mean_test_recall,mean_test_sensitivity,mean_test_specificity,mean_test_precision,mean_test_NPV,mean_test_PPV
0,Test_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True_LCN_LCP,0.770416,0.766269,0.817495,0.741037,0.708982,0.708982,0.823556,0.778037,0.766453,0.778037
0,Test_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_Top_LCN_LCP,0.759692,0.758607,0.843298,0.741805,0.743598,0.743598,0.773616,0.740448,0.777003,0.740448
0,Test_T81noDag_Boruta_T1000_itrr500_thres100_topR5_MD3_True_LCN_LCP,0.729630,0.725180,0.817960,0.694367,0.662764,0.662764,0.787596,0.732189,0.730064,0.732189
0,Test_T81_Boruta_T1000_itrr200_thres100_topR5_MD3_True_LCN_LCP,0.729630,0.725180,0.817960,0.694367,0.662764,0.662764,0.787596,0.732189,0.730064,0.732189
0,Test_T81_Boruta_T1000_itrr200_thres100_topR5_MD3_Top_LCN_LCP,0.729625,0.725342,0.816219,0.695202,0.665170,0.665170,0.785515,0.731783,0.730938,0.731783
0,Test_T81noDag_Boruta_T1000_itrr500_thres100_topR5_MD3_Top_LCN_LCP,0.729625,0.725342,0.816219,0.695202,0.665170,0.665170,0.785515,0.731783,0.730938,0.731783
0,Test_T83_Boruta_T1000_itrr200_thres100_topR5_MD3_Top_LCN_LCP,0.726343,0.713939,0.774948,0.662521,0.616842,0.616842,0.811036,0.717103,0.732729,0.717103
0,Test_T83_Boruta_T1000_itrr200_thres100_topR5_MD3_True_LCN_LCP,0.714413,0.701439,0.763923,0.646121,0.600000,0.600000,0.802879,0.703620,0.722582,0.703620
0,Test_T81noDag_Boruta_T1000_itrr500_thres100_topR5_MD5_Top_LCN_LCP,0.709269,0.706220,0.803956,0.678488,0.662924,0.662924,0.749515,0.698305,0.720643,0.698305
0,Test_T83_Boruta_T1000_itrr200_thres100_topR5_MD7_Top_LCN_LCP,0.702448,0.689859,0.757975,0.634234,0.591579,0.591579,0.788138,0.687782,0.713922,0.687782


In [29]:
# dfTestResults.to_csv("OUTPUT/MP/05-classifiers/FS/FS_TestDatasetResults_T81_T83.csv", index=False)

## Summary - Best set so far seems to be Test_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True_LCN_LCP

# Holdoutset - validation testing - Boruta Tested Best Feature

In [ ]:
lScorersBinary = [
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    # "average_precision",
    "NPV",
    "PPV",
    # "neg_mean_squared_error",
]
lClassifiers = ["RF", "ET", "LogR"] # different classifiers
lCV = [3,5,10] # different cross-validation splits
lTrees=[100,1000,10000] #num trees
lMD=[None] #max depth


In [33]:
selFeat = "Test_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True_LCN_LCP".replace(
    "_LCN_LCP", ""
).replace("Test_", "")
lSelFeat = dFeat[selFeat]
x,y = getXY(T81_h)
x=x[lSelFeat]

In [47]:
dfResults = classify_MP(X=x, y=y, lScorers=lScorersBinary, lClassifiers=lClassifiers, lTrees=lTrees, lMD=lMD,lCV=lCV)

In [ ]:
# dfResults.columns

In [55]:
dfResults['Title']='R81_Holdout_'+selFeat

In [56]:
lResCol = ['Title','cv']+[x for x in dfResults.columns if x.startswith("param_") or x.startswith("mean_test_")]

In [71]:
dfResults[lResCol] # all results

,Title,cv,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,mean_test_accuracy,mean_test_balanced_accuracy,mean_test_roc_auc,mean_test_f1,mean_test_recall,mean_test_sensitivity,mean_test_specificity,mean_test_precision,mean_test_NPV,mean_test_PPV
0,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,3,RandomForestClassifier(),None,100,42,NaN,0.702455,0.699885,0.788684,0.674349,0.665841,0.665841,0.733930,0.684176,0.718477,0.684176
1,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,3,RandomForestClassifier(),None,1000,42,NaN,0.701390,0.698875,0.791620,0.673046,0.665793,0.665793,0.731958,0.681562,0.718642,0.681562
2,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,3,RandomForestClassifier(),None,10000,42,NaN,0.705681,0.703180,0.791043,0.677596,0.670422,0.670422,0.735938,0.686724,0.722897,0.686724
3,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,3,ExtraTreesClassifier(),None,100,42,NaN,0.692784,0.688157,0.730954,0.653469,0.626392,0.626392,0.749922,0.685168,0.700019,0.685168
4,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,3,ExtraTreesClassifier(),None,1000,42,NaN,0.694935,0.690631,0.728423,0.657253,0.633337,0.633337,0.747926,0.685721,0.703843,0.685721
5,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,3,ExtraTreesClassifier(),None,10000,42,NaN,0.696010,0.691629,0.727802,0.658020,0.633337,0.633337,0.749922,0.687405,0.704433,0.687405
6,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,3,LogisticRegression(),NaN,NaN,NaN,1000,0.770128,0.765683,0.840141,0.739753,0.705355,0.705355,0.826011,0.778178,0.764789,0.778178
0,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,5,RandomForestClassifier(),None,100,42,NaN,0.716405,0.713311,0.798193,0.685627,0.670623,0.670623,0.756000,0.706254,0.729044,0.706254
1,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,5,RandomForestClassifier(),None,1000,42,NaN,0.725007,0.721461,0.797399,0.692943,0.672922,0.672922,0.770000,0.717786,0.733738,0.717786
2,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,5,RandomForestClassifier(),None,10000,42,NaN,0.723943,0.720786,0.797618,0.693693,0.677573,0.677573,0.764000,0.712914,0.734625,0.712914


In [39]:
dfResults[
    (dfResults["cv"] == 5)
    # & (dfResults["param_clf"] == "RandomForestClassifier()")
    & (dfResults["param_clf__n_estimators"] == 1000)
][lResCol].T

,8,11
Title,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True
cv,5,5
param_clf,RandomForestClassifier(),ExtraTreesClassifier()
param_clf__max_depth,NaN,NaN
param_clf__n_estimators,1000.0,1000.0
param_clf__random_state,42.0,42.0
mean_test_accuracy,0.725007,0.715336
mean_test_balanced_accuracy,0.721461,0.709272
mean_test_roc_auc,0.797399,0.751675
mean_test_f1,0.692943,0.669664


In [38]:
dfResults[(dfResults['cv']==5) & (dfResults['param_clf']=='RandomForestClassifier()') & (dfResults['param_clf__n_estimators']==1000)][lResCol]

,Title,cv,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,mean_test_accuracy,mean_test_balanced_accuracy,mean_test_roc_auc,mean_test_f1,mean_test_recall,mean_test_sensitivity,mean_test_specificity,mean_test_precision,mean_test_NPV,mean_test_PPV
8,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,5,RandomForestClassifier(),NaN,1000.0,42.0,0.725007,0.721461,0.797399,0.692943,0.672922,0.672922,0.77,0.717786,0.733738,0.717786


In [79]:
dfResults.to_csv(
    f"OUTPUT/MP/05-classifiers/Validation/CA_{'R81_Holdout_'+selFeat}.csv", index=False
)
dfResults[lResCol].to_csv(f"OUTPUT/MP/05-classifiers/Validation/CA_{'R81_Holdout_'+selFeat}_selCol.csv", index=False)

In [37]:
dfResults = pd.read_csv(
    f"OUTPUT/MP/05-classifiers/Validation/CA_{'R81_Holdout_'+selFeat}.csv"
)

### Feature Importance of Best Set


In [61]:
clf = RandomForestClassifier(n_estimators=1000,max_depth=None,random_state=42)
clf.fit(x,y)
feature_importances = clf.feature_importances_
feature_importances = pd.Series(feature_importances, index=x.columns).sort_values(
    ascending=False
)

In [62]:
feature_importances

age                                     0.419838
dag                                     0.235372
SYMPT-fatigue___2                       0.084360
SYMPT-dizziness___2                     0.079019
SYMPT-muscle_aches_myalgia___2          0.074982
SYMPT-shortness_of_breath_dyspne___2    0.059716
SYMPT-joint_pain_arthralgia___2         0.046712
dtype: float64

In [76]:
feature_importances.to_csv(f"OUTPUT/MP/05-classifiers/Validation/FR_{'R81_Holdout_'+selFeat}.csv", index=False)

age                                     0.419838
dag                                     0.235372
SYMPT-fatigue___2                       0.084360
SYMPT-dizziness___2                     0.079019
SYMPT-muscle_aches_myalgia___2          0.074982
SYMPT-shortness_of_breath_dyspne___2    0.059716
SYMPT-joint_pain_arthralgia___2         0.046712

In [65]:
clf = RandomForestClassifier(n_estimators=1000, max_depth=None, random_state=42)
cross_val_score(clf, x, y, cv=5, scoring="accuracy").mean()

0.7250071876257836